In [9]:
#encoding utf-8
import numpy as np
import pandas as pd
import gensim
from IPython.display import Image
import json
import urllib2
from time import sleep


In [10]:
NN_POS_PAIR = pd.read_pickle('./NN_POS_PAIR')
NV_POS_PAIR = pd.read_pickle('./NV_POS_PAIR')

In [ ]:
##########################################################################
# 要增加 pos 認識的 term 請在這裏
# ex: dff_soup 要多認識一個名詞 '濃稠度
#'data = { 'NPOS': np.array(['dft_soup']), 'NTERM': np.array(['濃稠度])}  
# NN_POS_PAIR = NN_POS_PAIR.append(pd.DataFrame(data),ignore_index=True)  


In [12]:
# Initialize word2vector model
model=gensim.models.Word2Vec.load_word2vec_format('../02_Word2Vector/bin/mobile01_ptt.bin',binary=True)

In [ ]:
# 資料觀察用, 儲存 word2vector 失敗的的 term
ptt_no_data = []
for idx in range(len(NN_POS_PAIR)):
    NN_NPOS = NN_POS_PAIR['NPOS'][idx]
    NN_NTERM= NN_POS_PAIR['NTERM'][idx]
    #print NN_NPOS + '\t' + repr(NN_NTERM).decode('unicode-escape')
    try:
        w2v_result = model.most_similar(NN_NTERM)
        
    except KeyError:
        # ERROR HANDLEING...SAVE THOSE TERM AS KNOWLEDGE
        print 'Oops!  [' + str(idx) +']\t' + repr(NN_NTERM).decode('unicode-escape') + 'is not work in word2vector'
        ptt_no_data.append(NN_NTERM)
        continue
        
    # FOR DATA OBSERVATION...
    pp = pd.DataFrame(np.array(w2v_result))
    pos_numpy = pd.DataFrame.as_matrix(pp)
    DATA_STATISTIC_RESHAPE = pd.DataFrame(pos_numpy.reshape(2,-1))
    #print DATA_STATISTIC_RESHAPE  
    
#儲存 failed term
# PTT_NO_DATA_LIST = pd.DataFrame(ptt_no_data)
# PTT_NO_DATA_LIST.to_pickle('./PTT_NO_DATA_N-LIST')

In [ ]:
# 延伸所有的 n
exception_case = [u'鼎泰豐',u'瘦肉精']
ptt_no_data = []
NN_POS_PAIR_EXT = NN_POS_PAIR
for idx in range(len(NN_POS_PAIR)):
    NN_NPOS = NN_POS_PAIR['NPOS'][idx]
    NN_NTERM= NN_POS_PAIR['NTERM'][idx]
    #有些已知 word2vector找出來效果很差的就跳過
    if NN_NTERM in exception_case:
        continue
    #print NN_NPOS + '\t' + repr(NN_NTERM).decode('unicode-escape')
    try:
        w2v_result = model.most_similar(NN_NTERM)
        
    except KeyError:
        # 還是會有錯誤的,但是這裏就不處理了
        #print 'Oops!  [' + str(idx) +']\t' + repr(NN_NTERM).decode('unicode-escape') + 'is not work in word2vector'
        #ptt_no_data.append(NN_NTERM)
        continue
    # Expansion data
    w2v_np_result = np.array(w2v_result)
    w2v_np_result = w2v_np_result[:,0] #刪除成績部分,只留下term
    data = {'NPOS': np.array([NN_POS_PAIR['NPOS'][idx]]*10),'NTERM':w2v_np_result}
    NN_POS_PAIR_EXT = NN_POS_PAIR_EXT.append(pd.DataFrame(data),ignore_index=True)  
    #print DATA_STATISTIC_RESHAPE

In [ ]:
# 延伸所有的 V
exception_case = [u'鼎泰豐',u'瘦肉精']
ptt_no_data = []
NV_POS_PAIR_EXT = NV_POS_PAIR
for idx in range(len(NV_POS_PAIR)):
    NV_NPOS = NV_POS_PAIR['NPOS'][idx]
    NV_VTERM= NV_POS_PAIR['VTERM'][idx]
    #有些已知 word2vector找出來效果很差的就跳過
    if NV_VTERM in exception_case:
        continue
    #print NN_NPOS + '\t' + repr(NN_NTERM).decode('unicode-escape')
    try:
        w2v_result = model.most_similar(NV_VTERM)
        
    except KeyError:
        # 還是會有錯誤的,但是這裏就不處理了
        #print 'Oops!  [' + str(idx) +']\t' + repr(NN_NTERM).decode('unicode-escape') + 'is not work in word2vector'
        #ptt_no_data.append(NN_NTERM)
        continue
    # Expansion data
    w2v_np_result = np.array(w2v_result)
    w2v_np_result = w2v_np_result[:,0] #刪除成績部分,只留下term
    data = {'NPOS': np.array([NV_POS_PAIR['NPOS'][idx]]*10),'VTERM':w2v_np_result}
    NV_POS_PAIR_EXT = NV_POS_PAIR_EXT.append(pd.DataFrame(data),ignore_index=True) 

In [ ]:
#刪除重複元素909->861,然後儲存
print len(NN_POS_PAIR_EXT)
NN_POS_PAIR_EXT = NN_POS_PAIR_EXT.drop_duplicates()
print len(NN_POS_PAIR_EXT)
NN_POS_PAIR_EXT.sort(['NPOS','NTERM'],ascending=[1,1])
NN_POS_PAIR_EXT.to_pickle('./NN_POS_PAIR_EXT') 

print len(NV_POS_PAIR_EXT)
NV_POS_PAIR_EXT = NV_POS_PAIR_EXT.drop_duplicates()
print len(NV_POS_PAIR_EXT)
NV_POS_PAIR_EXT.sort(['NPOS','VTERM'],ascending=[1,1])
NV_POS_PAIR_EXT.to_pickle('./NV_POS_PAIR_EXT') 

In [ ]:
#############################################################################################################
# 以下爲測試code
#############################################################################################################

In [18]:
NN_POS_PAIR['NPOS']

0             dtfsoup
1      dtfbranchkindb
2          ingredient
3         dtffoodkind
4            dtfbrand
5       dtffoodsaftya
6         dtffoodkind
7        dtfappetizer
8       dtffoodsaftyb
9       dtfingredient
10         ingredient
11         ingredient
12                  n
13                  n
14     dtfbranchkinda
15         ingredient
16         ingredient
17      dtffoodsaftyb
18         ingredient
19        dtffoodtype
20      dtfingredient
21        dtffoodkind
22        dtffoodtype
23          dtfbranch
24        dtffoodtype
25      dtffoodsaftya
26        dtffoodkind
27      dtffoodsaftyb
28                  n
29      dtffoodsaftya
            ...      
79            dtfsoup
80            dtfsoup
81           dtfbrand
82           dtfbrand
83           dtfbrand
84      dtfingredient
85         ingredient
86         ingredient
87         ingredient
88      dtfingredient
89        dtffoodkind
90       dtfappetizer
91      dtfingredient
92         ingredient
93        

In [17]:
# 單一 term 測試 word2vector
idx = 5
NN_NPOS = NN_POS_PAIR['NPOS'][idx]
NN_NTERM= NN_POS_PAIR['NTERM'][idx]
print NN_NPOS + '\t' + repr(NN_NTERM).decode('unicode-escape')
w2v_result = model.most_similar(NN_NTERM)
pp = pd.DataFrame(np.array(w2v_result))
pos_numpy = pd.DataFrame.as_matrix(pp)
DATA_STATISTIC_RESHAPE = pd.DataFrame(pos_numpy.reshape(2,-1))
DATA_STATISTIC_RESHAPE
#exception_case = ['鼎泰豐']

dtffoodsaftya	u'瘦肉精'


,0,1,2,3,4,5,6,7,8,9
0,安內,0.501488089561,乳源,0.499897748232,無菌,0.481448233128,和氣生財,0.480949193239,新聞稿,0.480463385582
1,黑肚,0.479298591614,管太多,0.475521743298,口水戰,0.471942275763,thinking,0.470590233803,編入,0.469600766897


In [ ]:
#測試延伸 data frame
w2v_np_result = np.array(w2v_result)
w2v_np_result = w2v_np_result[:,0] #刪除成績部分,只留下term
data = {'NPOS': np.array(['fooe']*10),'NTERM':w2v_np_result}
#data = {'NPOS': np.array([NN_POS_PAIR['NPOS'][0]]*10),'NTERM':w2v_np_result}

NN_POS_PAIR.append(pd.DataFrame(data),ignore_index=True)
#pd.DataFrame(data)


In [ ]:
# 對 dataframe做 reshape (10,2) --> (2,10)
pos_numpy = pd.DataFrame.as_matrix(pp)
DATA_STATISTIC_RESHAPE = pd.DataFrame(pos_numpy.reshape(2,-1))
DATA_STATISTIC_RESHAPE

In [ ]:
NN_POS_PAIR['NPOS'][0]
id

In [ ]:
len(NN_POS_PAIR_EXT)

In [ ]:
NN_POS_PAIR_EXT = NN_POS_PAIR_EXT.drop_duplicates()
NN_POS_PAIR_EXT.sort(['NPOS','NTERM'],ascending=[1,1])

In [6]:
NN_POS_PAIR_EXT = pd.read_pickle('./NN_POS_PAIR_EXT')
NV_POS_PAIR_EXT = pd.read_pickle('./NV_POS_PAIR_EXT')

In [ ]:
NN_POS_PAIR_EXT

In [ ]:
NN_POS_PAIR